# **Encuesta Financiera de las Familias (EFF) - España, 2022**

- Actualizar a las fuentes de datos más recientes.
- Para cada sección tenemos que nutrirnos de las 3 fuentes de datos.
- **Dibujar primero la pregunta antes del análisis.**
    - Por ej. Cliente: detectado 3 variables con edad, nivel educativo y estado civil.


### Descarga y preparación de los datos
Para empezar a trabajar con los microdatos de la EFF primero debemos descargarlos de la sección Microdatos y los guardaremos en la carpeta que queramos del explorador de archivos.

Para replicar los resultados del Artículo Analítico (AA) correspondiente a la EFF2022 utilizaremos las bases databol.dta en la que ya están creadas las variables que se muestran en los cuadros del AA (consultar el documento “definiciones” para ver cómo se construyen las variables del AA partiendo de las preguntas del cuestionario).

Para crear la base de datos, debemos unir las 5 imputaciones, y crear un indicador (variable implicate) que identifique cada una.

In [18]:
import pandas as pd
import numpy as np

# load the data creating the imputation indicator, using the correct separator
df_eff = pd.concat(
    [pd.read_csv(
        r"C:\Users\maria\Github repos\repos_externos\ProjecteData\Equip_9\Data\EFF2022\databol_2022_csv\databol{}.csv".format(i),
        sep=";"
    ).assign(imputation=i) for i in range(1, 6)]
)

# replace the values for legibility
replace_dict = {
    "bage": {
        1: "Under 35", 2: "35-44", 3: "45-54", 4: "55-64", 5: "65-74", 6: "Over 75"
    },
    "percrent": {
        1: "< P20", 2: "P20-P40", 3: "P40-P60", 4: "P60-80", 5: "P80-P90", 6: "> P90"
    },
    "nsitlabdom": {
        1: "Employee", 2: "Self-Employed", 3: "Retired", 4: "Other Inactive or Unemployed"
    },
    "neducdom": {
        1: "Below Secondary Education", 2: "Secondary Education", 3: "University Education"
    },
    "np2_1": {
        1: "Ownership", 2: "Other"
    },
    "nnumadtrab": {
        0: "None", 1: "One", 2: "Two", 3: "Three or More"
    },
    "np1": {
        5: "5 or more"
    },
    "percriq": {
        1: "< P25", 2: "P25-P50", 3: "P50-P75", 4: "P75-P90", 5: "> P90"
    }
}
df_eff["np1"] = df_eff["np1"].astype(int)
df_eff = df_eff.replace(to_replace=replace_dict)

In [19]:
df_eff

,h_2022,facine3,p2_69,p2_84,p2_70,p2_71,p4_7_3,p4_15,p4_24,p4_35,...,riquezanet,percriq,pagodeuda,alim,nodur,gvehic,gimpvehic,tvehic,timpvehic,imputation
0,1,2956.913200,1,100000.0,1000.0,500000,7000.0,15000.0,NaN,10000.0,...,1350530.0,> P90,0,14400.0,13212.0,0,0,1,6500,1
1,2,50.462724,2,30000.0,NaN,90000,125000.0,600000.0,NaN,75000.0,...,3805000.0,> P90,0,7200.0,15600.0,0,0,1,30000,1
2,3,67.518567,1,30000.0,5000.0,40000,24000.0,NaN,NaN,NaN,...,3414000.0,> P90,0,9600.0,38400.0,0,0,1,20000,1
3,4,7216.692200,1,NaN,600.0,8000,17000.0,NaN,NaN,NaN,...,158471.0,P50-P75,47,10800.0,7200.0,1,10000,1,21000,1
4,5,3696.269000,2,NaN,NaN,6000,240.0,NaN,NaN,NaN,...,177538.5,P50-P75,150,1200.0,1368.0,1,120,1,580,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6380,6381,7648.026800,1,4000.0,1000.0,30000,8000.0,NaN,NaN,NaN,...,212000.0,P50-P75,0,4800.0,2160.0,0,0,1,2000,5
6381,6382,64.297133,2,100000.0,NaN,60000,70000.0,1000000.0,NaN,NaN,...,4895050.0,> P90,0,24000.0,36000.0,0,0,1,20000,5
6382,6383,2793.780400,1,1500.0,5000.0,12000,250.0,NaN,NaN,NaN,...,43050.0,P25-P50,270,7200.0,8400.0,0,0,0,0,5
6383,6384,3333.792800,1,3000.0,2000.0,20000,25000.0,350.0,NaN,NaN,...,28350.0,< P25,0,6000.0,11580.0,1,30000,1,55500,5


### Cálculo de estadísticos con pesos e imputaciones
- ##### Medias y medianas para el total de hogares
Esta sección muestra cómo calcular medias, proporciones y medianas con la EFF. En concreto, se replican los resultados del Cuadro 1A del AA, que muestra la media y la mediana de la renta de los hogares (renthog21_eur22)

Hay que tener en cuenta que el diseño estratificado con sobrerrepresentación del nivel de riqueza de la muestra hace que sea necesario utilizar los pesos (variable facine3) para el cálculo de estadísticos descriptivos.

Además, para estimar estadísticos de posición (como lo son la media o la mediana u otros percentiles de la distribución) utilizando las 5 imputaciones:
-   Primero debemos calcular el estadístico deseado por separado en cada una de las 5 imputaciones.
-   Posteriormente debemos calcular la media simple de las 5 estimaciones:

In [20]:
# create a function to calcualte the weighted median
def weighted_median(variable, weights):
    variable = variable.values
    weights = weights.values
    sorted_idx = np.argsort(variable)
    cum_weights = np.cumsum(weights[sorted_idx])
    lower_percentile_idx = np.searchsorted(
        cum_weights, 0.5 * cum_weights[-1]
    )
    return variable[sorted_idx[lower_percentile_idx]]

# use a group by operation to calculate the weighted statistic for each implicate, 
# and then average over the 5 implicates

mean_renthog = df_eff.groupby('imputation').apply(lambda x: np.average(
    x['renthog21_eur22'], weights=x['facine3'])).mean()
print("Mean: {:.2f}".format(mean_renthog))
median_renthog = df_eff.groupby('imputation').apply(lambda x: weighted_median(
    variable=x['renthog21_eur22'], weights=x['facine3'])).mean()
print("Median: {:.2f}".format(median_renthog))

Mean: 41764.02
Median: 31595.66


C:\Users\maria\AppData\Local\Temp\ipykernel_21048\3001756785.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mean_renthog = df_eff.groupby('imputation').apply(lambda x: np.average(
C:\Users\maria\AppData\Local\Temp\ipykernel_21048\3001756785.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  median_renthog = df_eff.groupby('imputation').apply(lambda x: weighted_median(


- ##### Medias y medianas por subgrupos de población
Adicionalmente, es interesante conocer la media y mediana de la renta de los hogares por grupos poblacionales. Para calcularlo, debemos dividir la muestra en los subgrupos poblacionales para los que queramos hacer las estimaciones, y seguir los dos pasos previos (1. Estimación en cada imputación y 2. Media simple de las 5 estimaciones).

Comenzamos con una muestra de cómo calcular la media y mediana de la renta de los hogares (renthog21_eur22) por edad del cabeza de familia(bage)

In [21]:
# use a group by operation to calculate the weighted statistic for each implicate and breakdown category, 
# and then average over the 5 implicates

mean_renthog = df_eff.groupby(['imputation', 'bage']).apply(lambda x: np.average(
    x['renthog21_eur22'], weights=x['facine3'])).reset_index().pivot(values=0, columns='bage', index='imputation').mean(0)
print("Mean income by household head age:", mean_renthog)
median_renthog = df_eff.groupby(['imputation', 'bage']).apply(lambda x:  weighted_median(
    variable=x['renthog21_eur22'], weights=x['facine3'])).reset_index().pivot(values=0, columns='bage', index='imputation').mean(0)
print("Median income by household head age:", median_renthog)

Mean income by household head age: bage
35-44       42695.832584
45-54       45605.325419
55-64       47528.409887
65-74       40853.498727
Over 75     29691.687826
Under 35    33164.719221
dtype: float64
Median income by household head age: bage
35-44       34172.4490
45-54       36152.8200
55-64       34696.3476
65-74       28982.0878
Over 75     20773.7062
Under 35    28435.9900
dtype: float64


C:\Users\maria\AppData\Local\Temp\ipykernel_21048\3274701454.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mean_renthog = df_eff.groupby(['imputation', 'bage']).apply(lambda x: np.average(
C:\Users\maria\AppData\Local\Temp\ipykernel_21048\3274701454.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  median_renthog = df_eff.groupby(['imputation', 'bage']).apply(lambda x:  weighted_median(


- ##### Proporciones
Un cálculo adicional que aparece en el Cuadro 1.A es el porcentaje de hogares que forman parte de cada uno de los grupos de edad. Una proporción puede estimarse como la media de una variable binaria que tome valor 1 si una observación forma parte de dicho subgrupo, y 0 si no forma parte. Por lo tanto, podemos calcular proporciones construyendo variables binarias y replicando los pasos anteriores para el cálculo de la media.

In [22]:
prop = df_eff.groupby(['imputation', 'bage'])["facine3"].sum().reset_index().pivot(
    values="facine3", index='imputation', columns='bage').mean(0)
prop = (prop / prop.sum()) * 100
print("Proportion of households by household head age:", prop)

Proportion of households by household head age: bage
35-44       18.202284
45-54       24.526779
55-64       21.398277
65-74       15.675929
Over 75     13.409894
Under 35     6.786836
dtype: float64


- ##### Réplica del cuadro 1A del AA
Juntando todo lo anterior, el siguiente código replica el cuadro 1A del AA, calculando la proporción de hogares en cada subgrupo poblacional, y la la media y mediana de la renta para todos los hogares y todos los subgrupos que se consideran en del artículo analítico:

In [23]:
groups = list(replace_dict.keys())
groups.extend([None])

# let's create a function to parametrise the calculation of any given variable statistic

def get_group_statistics(variable, group_name):

    # variable: str
    #   indicates the name of the variable. i.e.: "riquezanet"
    # group_name: str
    #   indicates the name of the breakdown. i.e.: "bage"

    if group_name is not None:
        mean_var = df_eff.groupby(['imputation', group_name]).apply(lambda x: np.average(
            x[variable], weights=x['facine3'])).reset_index().pivot(
            values=0, columns=group_name, index='imputation').mean(0).rename("Mean")
        median_var = df_eff.groupby(['imputation', group_name]).apply(lambda x: weighted_median(
            variable=x[variable], weights=x['facine3'])).reset_index().pivot(values=0, columns=group_name,
                                                                             index='imputation').mean(0).rename("Median")
        prop_var = df_eff.groupby(['imputation', group_name])["facine3"].sum().reset_index().pivot(
            values="facine3", index='imputation', columns=group_name).mean(0)
        prop_var = (prop_var / prop_var.sum()) * 100
        prop_var = prop_var.rename("Proportion")

        df = pd.concat([mean_var, median_var, prop_var], axis=1).stack().reset_index()
        df.columns = ['category', "statistic", "value"]
        df['variable'] = variable
        df['group'] = group_name
    else:
        mean_var = df_eff.groupby('imputation').apply(lambda x: np.average(
            x[variable], weights=x['facine3'])).mean()
        median_var = df_eff.groupby('imputation').apply(lambda x: weighted_median(
            variable=x[variable], weights=x['facine3'])).mean()
        prop = 100
        df = pd.DataFrame({
            "category": "All Households",
            "statistic": ["Mean", "Median", "Proportion"],
            "value": [mean_var, median_var, prop],
            "variable": "All Households",
            "group": "All Households"
        })

    return df


table_1A = [get_group_statistics("renthog21_eur22", group) for group in groups]
table_1A = pd.concat(table_1A)

C:\Users\maria\AppData\Local\Temp\ipykernel_21048\558177938.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mean_var = df_eff.groupby(['imputation', group_name]).apply(lambda x: np.average(
C:\Users\maria\AppData\Local\Temp\ipykernel_21048\558177938.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  median_var = df_eff.groupby(['imputation', group_name]).apply(lambda x: weighted_median(
C:\Users\mari